In [158]:
from decimal import Decimal
from string import ascii_lowercase

class BankAccount:
    _monthly_interest_rate = None
    def __init__(self, accnt_no, f_name, l_name, tz_offset=0, balance=0):
        self.accnt_no = accnt_no
        self.f_name = f_name
        self.l_name = l_name
        self.tz_offset = tz_offset
        self.balance = balance
        
    @classmethod
    def get_interest_rate(cls):
        if cls._monthly_interest_rate is None:
            print('Interest rate has not been set.')
        return cls._monthly_interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        try:
            value = Decimal(value)
        except:
            raise ValueError('Interest rate must be a decimal value.')
        cls._monthly_interest_rate = value
        
    @property
    def accnt_no(self):
        return self._accnt_no
    
    @accnt_no.setter
    def accnt_no(self, value):
        try:
            value = int(value)
        except:
            raise ValueError('Account number must be an integer value.')
        self._accnt_no = value
        
    @property
    def f_name(self):
        return self._f_name
    
    @f_name.setter
    def f_name(self, value):
        # If every character in the provided value (assuming it's a string) is not a
        # standard letter, a space, or an apostrophe, raise an exception.
        if not all(map(lambda x: x.lower() in ascii_lowercase + " '", value)):
            raise ValueError('First name must contain only letters, spaces, or apostrophes.')
        self._f_name = value
        
    @property
    def l_name(self):
        return self._l_name
    
    @l_name.setter
    def l_name(self, value):
        # If every character in the provided value (assuming it's a string) is not a
        # standard letter, a space, or an apostrophe, raise an exception.
        if not all(map(lambda x: x.lower() in ascii_lowercase + " '", value)):
            raise ValueError('Last name must contain only letters, spaces, or apostrophes.')
        self._l_name = value
        
    @property
    def full_name(self):
        return f'{self.f_name} {self.l_name}'

In [147]:
ba = BankAccount('12345', 't', 'z')

In [148]:
ba.accnt_no

12345

In [149]:
ba.accnt_no = 'hello'

ValueError: Account number must be an integer value.

In [150]:
ba.accnt_no

12345

In [151]:
ba.__dict__

{'_accnt_no': 12345,
 '_f_name': 't',
 '_l_name': 'z',
 'tz_offset': 0,
 'balance': 0}

In [152]:
ba.l_name = 'Van der Lynden'

In [153]:
ba.f_name = "O'Shaughnessey"

In [154]:
ba.l_name

'Van der Lynden'

In [155]:
ba.f_name

"O'Shaughnessey"

In [156]:
ba.full_name

"O'Shaughnessey Van der Lynden"

In [157]:
all(map(lambda x: x.lower() in ascii_lowercase + " '", test_string))

True